In [ ]:
!sudo apt install tesseract-ocr --quiet
!pip install pytesseract --quiet
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet

Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [ ]:
from pyngrok import ngrok

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

--2021-07-29 18:31:47--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75482 (74K) [text/plain]
Saving to: ‘haarcascade_russian_plate_number.xml.5’

haarcascade_russian 100%[===================>]  73.71K  --.-KB/s    in 0.01s   

2021-07-29 18:31:47 (5.52 MB/s) - ‘haarcascade_russian_plate_number.xml.5’ saved [75482/75482]



In [ ]:
%%writefile app.py

"""cv2 and streamlit don't work together. Istead, PIL is used.
And we require one of them to read our image so as to work over it. The upoaded image 'i' cannot be used directly.
"""
import pytesseract
import streamlit as st
from PIL import Image
import cv2
import numpy as np

st.title("Number Plate Detector")
i= st.sidebar.file_uploader("Choose an image:")  #'sidebar' moves our browzer to the left side of the website page.

if i is not None:    #Need to write 'if' since streamlit works in loops and until it receives the image we have uploaded, it will show an error saying no image found.
  st.image(i, caption= 'YOUR IMAGE') #displaying image.
  img= Image.open(i) #reading image so as to be able to use it.
  img= np.array(img)  #since OpenCV works in numpy arrays.

  if st.button('DETECT'):    #this if creates the button 'DETECT' and shows the required output only if that button is pressed.
    model= cv2.CascadeClassifier('/content/haarcascade_russian_plate_number.xml')
    plate= model.detectMultiScale(img, 1.1, 5)
    count= 0

    for (x,y,w,h) in plate:
      count += 1
      p= img[y:y+h, x:x+w]
      st.image(p, caption= f'Number Plate {count}')
      cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 5)
      cv2.putText(img, f'Number plate {count}', (int((x+x+w)/4),y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)

      text= pytesseract.image_to_string(p)
      st.text(f"Number {count}:")
      st.write(text)
    st.image(img, caption= f'Total Number Plates Detected')

Overwriting app.py


In [ ]:
!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)

nohup: appending output to 'nohup.out'
http://bb9b8fe7aa63.ngrok.io
